In [2]:
import pandas as pd
import re
import glob
import json, csv
from nltk.tokenize import word_tokenize, SpaceTokenizer, WhitespaceTokenizer

    
def get_files_list():
    
    path = '/home/ayushi/Documents/coursera&DS/HCL ML Challenge2/HCL ML Challenge Dataset/'
    fname_list = [i for i in glob.glob(path+'*.txt')]
        
    return fname_list


def get_2019_ind(fname):
    
    file = pd.read_csv(fname, sep='\n', header=None)

    file[0] = file[0].str.strip() 
    file[0] = file[0].apply(lambda x: ' '.join(x.split())) 

    for i in range(len(file[0])):
        if re.match(r'[0-9a-zA-Z\s]*2019[0-9a-zA-Z\s]*', file[0][i]) and re.match(r'\w*\s*[\£\€]+\s*',file[0][i+1]):
            if file[0][i].split()[-1]=='2019':
                # 2019 yr is present at last location
                flag = -1
            elif file[0][i].split()[-2]=='2019':
                # 2019 yr is present at 2nd last location
                flag = -2
            listlength = -1111
            break
        elif not re.match(r'[0-9a-zA-Z\s]*2019[0-9a-zA-Z\s]*', file[0][i]) and re.match(r'\w*\s*[\£\€]+\s*',file[0][i+1]):
            flag = -9999
            # 2019 is not present
            listlength = len(file[0][i].split())
            if (re.search(r'\s*Notes\s*', file[0][i], re.I) and listlength==2):
                listlength = 1
            break
        else:
            continue
    
    file[0] = file[0][i+2:]
    file = file[~file[0].isnull()][:18]            

    return file, flag, listlength
    

def extract_values():
    
    df = pd.DataFrame(index=range(500), columns=["Filename", "Extracted Values"])
    wst = WhitespaceTokenizer()
    
    file_list = get_files_list()
    pos = -1
    strflag = 0
    
    for name in file_list:
        #print(name)
        file, flag, listlen = get_2019_ind(name)

        strjoin = ''
        val_dict = {}
        pos += 1
        
        if flag != -9999:
            ind = flag
        else:
            if listlen >= 2:
                ind = -2
            else:
                ind = -1
                
        for line in file[0]:
            line = line.strip()
            #print('line: ', line)
            
            if (('Number' in line) or ('Act' in line) or ('Companies' in line) or ('Approved' in line) or ('And' in line) \
                or ('Statements' in line) or ('STATEMENTS' in line) or ('NOTES' in line) or ('Notes' in line) or ('London' in line) \
                or ('information' in line) or ('Broadfields' in line) or ('Southampton' in line)):
                break

            wrd_list = wst.tokenize(line)       

            if line=='year':
                key = strjoin + line
                strjoin = ''
                if flag == -9999:
                    val_dict[key] = "nan"
                else:
                    value = value.replace('(','-').replace(')','').replace(',','') if not ('{' in value or '}' in value) else value
                    try:
                        val_dict[key] = str(int(float(value)))
                    except:
                        val_dict[key] = value
                continue

            if len(wrd_list)<=12 and len(wrd_list) >= 2:
                    if re.match(r'^\d+[a-zA-z0-9\£\€\s]*', line) and line != '0 0':
                        if (not any(i.isdigit() for i in wrd_list[ind])) or (re.search(r'£',wrd_list[ind])):
                            key = ' '.join(wrd_list)
                            if flag != -9999:
                                val = "nan"
                        elif flag != -9999:
                            try:
                                sent = wrd_list[:ind] 
                                if sent[-1].isdigit():
                                    sent = sent[:-1]
                                key = ' '.join(sent)
                                val = wrd_list[ind]    
                            except:
                                key = ' '.join(wrd_list[:ind+1])
                                val = wrd_list[ind+1]   
                        elif flag == -9999:
                            key = ' '.join(wrd_list[:ind])
                            val = "nan"
                            
                        key = key.replace('€', '&#8364').replace('£', '&#163')
                        if key and not all(i.isdigit() for i in key if i!=','):
                            val_dict[key] = val
                            continue
                    
                    if re.match(r'\(*\{*[-0-9,]+\)*',wrd_list[ind]) and re.match(r'\(*[a-zA-Z:&-]+\)*', wrd_list[0]) and wrd_list[0] != '-':
                        sent = ' '.join(i for i in wrd_list[:ind] if not i.isdigit())
                        key = strjoin + sent
                        if sent and re.match(r'\(*\{*[-0-9,]+\)*', sent.split()[-1]):
                            sent = sent.split()[:-1]
                            key = strjoin + ' '.join(i for i in sent)

                        if flag != -9999:
                            try:
                                val = wrd_list[ind].replace('(','-').replace(')','').replace(',','') if not ('{' in wrd_list[ind] or '}' in wrd_list[ind]) else wrd_list[ind]
                            except:
                                val = wrd_list[ind]  
                        else:
                            val = "nan"
                        strjoin = ''
                    
                    elif re.match(r'\(*[-0-9,]+\)*', wrd_list[ind]) and not re.match(r'\(*[a-zA-Z:&]+\)*', wrd_list[0]):
                        if flag != -9999:
                            value = wrd_list[ind]  
                            if not strflag:
                                val_dict['nan'] = value
                        else:
                            value = "nan"
                        continue
                    
                    elif not re.match(r'\(*[-0-9,]+\)*', wrd_list[ind]) and re.match(r'\(*[a-zA-Z:&-]+\)*', wrd_list[0]) \
                    and ((re.search(r'\s*assets\s*', line, re.I)) or (re.search(r'\s*capital\s*', line, re.I) and not re.search(r'\s*classed\s*', line))):
                        key = ' '.join(wrd_list)
                        val = "nan"
                    
                    else:
                        strjoin = ' '.join(wrd_list) + ' '
                        strflag = 1
                        continue
            
            else:
                if re.match(r'\(*[-0-9,]+\)*', wrd_list[ind+1]) and not re.match(r'\(*[a-zA-Z:&]+\)*', wrd_list[0]) and flag != -9999:
                    if wrd_list[ind+1] != '-':
                        value = wrd_list[ind+1]
                    else:
                        value = "nan"
                elif not re.match(r'\(*[-0-9,]+\)*', wrd_list[ind+1]) and re.match(r'\(*[a-zA-Z:&-]+\)*', wrd_list[0]) \
                    and ((re.match(r'assets', line, re.I)) or (re.match(r'capital', line, re.I))):
                    key = ' '.join(wrd_list)
                    val = "nan"
                    
            if key[0].isupper() or (key[0].isdigit() and flag == -9999):
                try:
                    val_dict[key] = str(int(float(val)))
                except:
                    val_dict[key] = val

        #print(val_dict,'\n')
        
        extracted_data(val_dict, name, pos, df)
    df['Extracted Values'] = df['Extracted Values'].apply(lambda x: str(x).replace("'", '\"'))
    df.to_csv('resultfile.csv', index=False)
    
    #res = pd.read_csv('resultfile.csv')
    #print(df.head(50))
    
def extracted_data(dictnry, filename, pos, df):
    
    df["Filename"][pos] = re.findall(r'([A-Z0-9]+)\.txt$', filename)[0]
    df["Extracted Values"][pos] = dict(dictnry)
    
    
extract_values()
                                 